In [1]:
# Load environment variables and set up auto-reload
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload

In [2]:
import sys
sys.path.append('../')

In [3]:
from src.utils import show_prompt
from src.prompt import supervisor_decision_to_route_to_subagents
show_prompt(supervisor_decision_to_route_to_subagents, "supervisor_decision_to_route_to_subagents")

c:\Users\kamalmo\Desktop\AI\InboundLogisticSystem\MultiAgentInboundLogistic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


╭─────────────────────────────────── supervisor_decision_to_route_to_subagents ───────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  These are the inbound logistics Data received so far:                                                          │
│  <message>                                                                                                      │
│  {message}                                                                                                      │
│  </message>                                                                                                     │
│                                                                                                                 │
│  Today's date is {date}.                                                                                        │
│                                                                                                                 │
│  Your role is to act as the Supervisor Agent in the Inbound Logistics system.                                   │
│  Your responsibilities are:                                                                                     │
│  1. Assess the data provided by the user.                                                                       │
│  2. Decide whether the task should be delegated to:                                                             │
│     - `logistics_agent` → if the request relates to the following fields: {logistics_fields}. **Ensure any      │
│  provided data adheres to their `dataType` and `seededValues`, if provided.**                                   │
│     - `forwarder_agent` → if the request relates to the following fields: {forwarder_fields}. **Ensure any      │
│  provided data adheres to their `dataType` and `seededValues`, if provided.**                                   │
│  3. If the request is ambiguous or missing critical details, ask the user a **clarifying question** before      │
│  assigning the task.                                                                                            │
│  4. **Do not enforce field entry yourself — instead, delegate field-specific responsibilities to the            │
│  appropriate sub-agent (`logistics_agent` or `forwarder_agent`) based on {logistics_fields} and                 │
│  {forwarder_fields}.**                                                                                          │
│                                                                                                                 │
│  Guidelines for asking clarification:                                                                           │
│  - Only ask if **absolutely necessary**.                                                                        │
│  - Keep questions **concise and structured**. Use bullet points or lists if multiple clarifications are         │
│  needed.                                                                                                        │
│  - Do not repeat questions if the information is already provided.                                              │
│                                                                                                                 │
│  Respond in **valid JSON format** with these exact keys:                                                        │
│  - `"question"` : "<clarifying question used with <delegate_to=clarify_with_user> if necessary information is   │
│  needed, otherwise empty>"                                                                                      │
│  - `"delegate_to"` : "logistics_agent" | "forwarder_agent" | "supervisor_tools" | "clarify_with_user"           │
│  - `"agent_brief"`: "<acknowledgement message confirmi

In [4]:
%%writefile ../src/supervisor_schema.py

"""Supervisor Definitions and Pydantic Schemas for Routing Workflow.

This defines the state objects and structured schemas used for Routing to 
sub agent sworkflow, including Supervisor state management and output schemas.
"""

import operator
from typing_extensions import Optional, Annotated, List, Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from langchain_core.tools import tool, InjectedToolArg

from pydantic import BaseModel, Field
from enum import Enum

# ===== STRUCTURED OUTPUT SCHEMAS =====

class NextAgent(str, Enum):
    END = "__end__"
    CLARIFY_WITH_USER = "clarify_with_user"
    LOGISTICS_AGENT   = "logistics_agent"
    FORWARDER_AGENT   = "forwarder_agent"
    SUPERVISOR_TOOLS  = "supervisor_tools"
    
class ClarifyWithUser(BaseModel):
    """Schema for delegation decision and questions."""
    question: str = Field(
        description = "A question to ask the user to clarify the report scope",
    )
    delegate_to: NextAgent = Field(
        description = "A decision to delegate and route the task to the next agent",
    )
    agent_brief: str = Field(
        description = "A Brief that will be used to route the task to the next sub-agent",
    )
    
class AgentInputState(MessagesState):
    """Input state for the full agent - only contains messages from user input."""
    pass

class AgentState(MessagesState):
    """
    Main state for the full multi-agent system.
    
    Extends MessagesState with additional fields for routing coordination.
    """
    supervisor_messages: Annotated[Sequence[BaseMessage], add_messages]
    clarification_schemas: Optional[ClarifyWithUser] = None
    agent_brief: str

Overwriting ../src/supervisor_schema.py


In [5]:
%%writefile ../src/supervisor_agent.py

"""User Clarification and Routimg to Sub Agents.

This module implements the  the Routing workflow, where we:
1. Assess if the user's data needs clarification
2. Delegate and route to Sub Agents

The workflow uses structured output to make deterministic decisions about
whether sufficient context exists to proceed with Routing.
"""

import json
from dotenv import load_dotenv
from datetime import datetime
from typing_extensions import Literal

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage , get_buffer_string
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from langgraph.checkpoint.memory import InMemorySaver

from src.prompt import supervisor_decision_to_route_to_subagents
from src.supervisor_schema import AgentState, ClarifyWithUser, AgentInputState, NextAgent

# Load environment variables
load_dotenv()

checkpointer = InMemorySaver()


# ===== IBL FIELDS =====
try:
    with open ("../IBL_SCHEMA.json" , "r") as config_file:
        routing_fields = json.load(config_file)
except FileNotFoundError:
    print("Error: config.json not found. Please create it.")

# ===== UTILITY FUNCTIONS =====
def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %#d, %Y")

# Set up tools and model binding
tools = []
tools_by_name = {tool.name: tool for tool in tools}

# Initialize model
model = init_chat_model(model="openai:gpt-4.1", temperature=0.0)
model_with_tools = model.bind_tools(tools)


# ===== WORKFLOW NODES =====
def supervisor_agent(state: AgentState):
    """
        Supervisor Agent determines if the input data sufficient to make 
        deterministic decisions and assign the task to the next agent.
    """
    # Set up structured output model
    structured_output_model = model_with_tools.with_structured_output(ClarifyWithUser)

    # Invoke the model with clarification instructions
    response = structured_output_model.invoke([
        HumanMessage(content=supervisor_decision_to_route_to_subagents.format(
            message=get_buffer_string(messages=state["messages"]), 
            date=get_today_str(),
            logistics_fields=routing_fields.get("logistics_agent"),
            forwarder_fields=routing_fields.get("forwarder_agent")
        ))
    ])
    
    # Route based on clarification need
    return {
             "clarification_schemas" : response ,
             "agent_brief" : response.agent_brief,
             "supervisor_messages": [
                                    AIMessage(content=response.delegate_to.value)
                                    ]
           }

def supervisor_tools(state: AgentState):
    """
        Executes all tool calls from the Supervisor Agent response.
        Returns updated state with tool execution results.
    """
    tool_calls = state["supervisor_messages"][-1].tool_calls
 
    # Execute all tool calls
    observations = []
    for tool_call in tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observations.append(tool.invoke(tool_call["args"]))
            
    # Create tool message outputs
    tool_outputs = [
        ToolMessage(
            content=observation,
            name=tool_call["name"],
            tool_call_id=tool_call["id"]
        ) for observation, tool_call in zip(observations, tool_calls)
    ]
    
    return {"supervisor_messages": tool_outputs}

def clarify_with_user(state: AgentState):
    """In Case the user needs to be asked a clarifying question."""
    clarification_schemas = state.get("clarification_schemas")
    if clarification_schemas and clarification_schemas.question:
        question = clarification_schemas.question
    return {"messages": [AIMessage(content=question)]}

def DelegateNextAgent(state: AgentState) -> Literal["logistics_agent", "forwarder_agent", "supervisor_tools", "clarify_with_user"]:
    
    """ 
        A routing logic that uses the supervisor agent's responses to determine 
        which agent should be assigned the task next 
    """

    # Then check the routing decision
    clarification_schemas = state.get("clarification_schemas")
    if not clarification_schemas:
        return "__end__"

    if clarification_schemas.delegate_to == NextAgent.LOGISTICS_AGENT:
        return "logistics_agent"
    elif clarification_schemas.delegate_to == NextAgent.FORWARDER_AGENT:
        return "forwarder_agent"
    elif clarification_schemas.delegate_to == NextAgent.CLARIFY_WITH_USER:
        return "clarify_with_user"
    else:
        return "__end__"

def logistics_agent(state: AgentState):
    pass

def forwarder_agent(state: AgentState):
    pass

# ===== GRAPH CONSTRUCTION =====

# Build the scoping workflow
supervisor_agent_builder = StateGraph(AgentState, input_schema=AgentInputState)

# Add workflow nodes
supervisor_agent_builder.add_node("supervisor_agent"  , supervisor_agent)
supervisor_agent_builder.add_node("supervisor_tools"  , supervisor_tools)
supervisor_agent_builder.add_node("clarify_with_user" , clarify_with_user)
supervisor_agent_builder.add_node("logistics_agent"   , logistics_agent)
supervisor_agent_builder.add_node("forwarder_agent"   , forwarder_agent)

# Add workflow edges
supervisor_agent_builder.add_edge(START, "supervisor_agent")
supervisor_agent_builder.add_conditional_edges(
    "supervisor_agent",
     DelegateNextAgent,
    {
        "supervisor_tools" : "supervisor_tools"  , # execute tools,
        "clarify_with_user": "clarify_with_user" , # Provide final answer
        "logistics_agent"  : "logistics_agent" ,
        "forwarder_agent"  : "forwarder_agent"
    },
)
supervisor_agent_builder.add_edge("supervisor_tools", "supervisor_agent")
supervisor_agent_builder.add_edge("clarify_with_user", END)

# Compile the workflow
SupervisorAgent = supervisor_agent_builder.compile(checkpointer = checkpointer)

Overwriting ../src/supervisor_agent.py


In [6]:
# Compile with in-memory checkpointer to test in notebook
from IPython.display import Image, display
from src.supervisor_agent import SupervisorAgent

display(Image(SupervisorAgent.get_graph(xray=True).draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [8]:
from src.utils import format_message
from langchain_core.messages import HumanMessage
thread = {"configurable":{"thread_id":"1"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="I want to enter the AWB 12345 and AWB Date")]} , config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB 12345 and AWB Date', additional_kwargs={}, response_metadata={}, id='1ebf28b9-ccc7-45a0-af9a-2fc4cffd449f'),
  AIMessage(content='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', additional_kwargs={}, response_metadata={}, id='e66089d7-17c4-49da-97ee-53c2fdb9da3d')],
 'supervisor_messages': [AIMessage(content='clarify_with_user', additional_kwargs={}, response_metadata={}, id='166e06bd-c576-4ff8-a6d7-e4f062cdd873')],
 'clarification_schemas': ClarifyWithUser(question='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', delegate_to=<NextAgent.CLARIFY_WITH_USER: 'clarify_with_user'>, agent_brief=''),
 'agent_brief': ''}

In [9]:
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="Well the AWB is 12345 and AWB Date 72025-08-12")]}, config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB 12345 and AWB Date', additional_kwargs={}, response_metadata={}, id='1ebf28b9-ccc7-45a0-af9a-2fc4cffd449f'),
  AIMessage(content='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', additional_kwargs={}, response_metadata={}, id='e66089d7-17c4-49da-97ee-53c2fdb9da3d'),
  HumanMessage(content='Well the AWB is 12345 and AWB Date 72025-08-12', additional_kwargs={}, response_metadata={}, id='58b9cbf5-ebdf-45f7-b8db-18e9c44fcd6c'),
  AIMessage(content="The provided AWB Date '72025-08-12' appears to be in an invalid or future format. Could you please confirm the correct AWB Date in YYYY-MM-DD format?", additional_kwargs={}, response_metadata={}, id='62f95d8c-6186-4773-b601-5a3982bf9ccc')],
 'supervisor_messages': [AIMessage(content='clarify_with_user', additional_kwargs={}, response_metadata={}, id='166e06bd-c576-4ff8-a6d7-e4f062cdd873'),
  AIMessage(content='clarify_with_user', additional_kwarg

In [10]:
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="Well the AWB is 12345 and AWB Date 2025-08-12")]}, config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB 12345 and AWB Date', additional_kwargs={}, response_metadata={}, id='1ebf28b9-ccc7-45a0-af9a-2fc4cffd449f'),
  AIMessage(content='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', additional_kwargs={}, response_metadata={}, id='e66089d7-17c4-49da-97ee-53c2fdb9da3d'),
  HumanMessage(content='Well the AWB is 12345 and AWB Date 72025-08-12', additional_kwargs={}, response_metadata={}, id='58b9cbf5-ebdf-45f7-b8db-18e9c44fcd6c'),
  AIMessage(content="The provided AWB Date '72025-08-12' appears to be in an invalid or future format. Could you please confirm the correct AWB Date in YYYY-MM-DD format?", additional_kwargs={}, response_metadata={}, id='62f95d8c-6186-4773-b601-5a3982bf9ccc'),
  HumanMessage(content='Well the AWB is 12345 and AWB Date 2025-08-12', additional_kwargs={}, response_metadata={}, id='2595e79b-2a49-41bc-999c-227b0c019fff')],
 'supervisor_messages': [AIMessage(content='clari

In [12]:
thread = {"configurable":{"thread_id":"2"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="I want to enter these values AWB/BL: AWB123456, AWB/BL Date: 2025-01-15, Forwarder: DHL , Incoterm: CIF, Product Temperature: 2–8°C, Packing: Cartons, Shipping Temp: Ambient, Gel Pack Expiry Date: 2025-12-30, Handover to Clearance: 2025-01-16, Aggregation: Batch A, Notified FF Date: 2025-01-14, Green light – Date: 2025-01-13, Shipment Mode: Air, Logistic Comment: Handle with care, Remark: Priority shipment, and ASN Importation Date: 2025-01-17")]} , config=thread)
result

{'messages': [HumanMessage(content='I want to enter these values AWB/BL: AWB123456, AWB/BL Date: 2025-01-15, Forwarder: DHL , Incoterm: CIF, Product Temperature: 2–8°C, Packing: Cartons, Shipping Temp: Ambient, Gel Pack Expiry Date: 2025-12-30, Handover to Clearance: 2025-01-16, Aggregation: Batch A, Notified FF Date: 2025-01-14, Green light – Date: 2025-01-13, Shipment Mode: Air, Logistic Comment: Handle with care, Remark: Priority shipment, and ASN Importation Date: 2025-01-17', additional_kwargs={}, response_metadata={}, id='a3394d6b-8272-404d-a37a-4b849b273060'),
  HumanMessage(content='I want to enter these values AWB/BL: AWB123456, AWB/BL Date: 2025-01-15, Forwarder: DHL , Incoterm: CIF, Product Temperature: 2–8°C, Packing: Cartons, Shipping Temp: Ambient, Gel Pack Expiry Date: 2025-12-30, Handover to Clearance: 2025-01-16, Aggregation: Batch A, Notified FF Date: 2025-01-14, Green light – Date: 2025-01-13, Shipment Mode: Air, Logistic Comment: Handle with care, Remark: Priority s

In [13]:
thread = {"configurable":{"thread_id":"3"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="I want to enter the AWB/BL 123457 and AWB Date, skip, yes, proceed without providing it")]} , config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB/BL 123457 and AWB Date, skip, yes, proceed without providing it', additional_kwargs={}, response_metadata={}, id='1183cd8f-b7a0-4752-9441-ddaec2b0a030'),
  AIMessage(content='You have provided the AWB/BL number (123457) but chose to skip the AWB/BL Date, which is a required field. Please confirm if you want to proceed without the AWB/BL Date, or would you like to provide it now?', additional_kwargs={}, response_metadata={}, id='2a851f8b-a4ad-4388-8ea3-280942b559db')],
 'supervisor_messages': [AIMessage(content='clarify_with_user', additional_kwargs={}, response_metadata={}, id='eb964356-0404-4953-9426-4e4951d870d0')],
 'clarification_schemas': ClarifyWithUser(question='You have provided the AWB/BL number (123457) but chose to skip the AWB/BL Date, which is a required field. Please confirm if you want to proceed without the AWB/BL Date, or would you like to provide it now?', delegate_to=<NextAgent.CLARIFY_WITH_USER: 'clarify_wi

In [ ]:
thread = {"configurable":{"thread_id":"3"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="I want to enter the AWB/BL 123457 and AWB Date, skip, yes, proceed without providing it")]} , config=thread)
result

In [7]:
from src.utils import format_message
from langchain_core.messages import HumanMessage
thread = {"configurable":{"thread_id":"1"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="I want to enter the AWB 12345 and AWB Date")]} , config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB 12345 and AWB Date', additional_kwargs={}, response_metadata={}, id='1412bc76-895c-4200-96fb-02fd740c78e0'),
  AIMessage(content='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', additional_kwargs={}, response_metadata={}, id='d163020b-e10a-451b-bc82-38fdfa3f2e34')],
 'supervisor_messages': [AIMessage(content='clarify_with_user', additional_kwargs={}, response_metadata={}, id='156c0830-2dee-49ef-8187-5e849822595a')],
 'clarification_schemas': ClarifyWithUser(question='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', delegate_to=<NextAgent.CLARIFY_WITH_USER: 'clarify_with_user'>, agent_brief=''),
 'agent_brief': ''}

In [8]:
thread = {"configurable":{"thread_id":"1"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="Skip")]} , config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB 12345 and AWB Date', additional_kwargs={}, response_metadata={}, id='1412bc76-895c-4200-96fb-02fd740c78e0'),
  AIMessage(content='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', additional_kwargs={}, response_metadata={}, id='d163020b-e10a-451b-bc82-38fdfa3f2e34'),
  HumanMessage(content='Skip', additional_kwargs={}, response_metadata={}, id='31933bb7-b6de-4370-a711-b218fcd2a2d2'),
  AIMessage(content='You provided AWB number 12345 but chose to skip entering the AWB Date, which is required. Would you like to provide the AWB Date now, or should we proceed without it?', additional_kwargs={}, response_metadata={}, id='4ada157b-1dd6-4ee4-9b21-c1a328bf3c9b')],
 'supervisor_messages': [AIMessage(content='clarify_with_user', additional_kwargs={}, response_metadata={}, id='156c0830-2dee-49ef-8187-5e849822595a'),
  AIMessage(content='clarify_with_user', additional_kwargs={}, response_metadata={

In [9]:
thread = {"configurable":{"thread_id":"1"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="Skip")]} , config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB 12345 and AWB Date', additional_kwargs={}, response_metadata={}, id='1412bc76-895c-4200-96fb-02fd740c78e0'),
  AIMessage(content='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', additional_kwargs={}, response_metadata={}, id='d163020b-e10a-451b-bc82-38fdfa3f2e34'),
  HumanMessage(content='Skip', additional_kwargs={}, response_metadata={}, id='31933bb7-b6de-4370-a711-b218fcd2a2d2'),
  AIMessage(content='You provided AWB number 12345 but chose to skip entering the AWB Date, which is required. Would you like to provide the AWB Date now, or should we proceed without it?', additional_kwargs={}, response_metadata={}, id='4ada157b-1dd6-4ee4-9b21-c1a328bf3c9b'),
  HumanMessage(content='Skip', additional_kwargs={}, response_metadata={}, id='262aaf5d-a0b0-4c98-9358-4fcf758affe5'),
  AIMessage(content='You have provided AWB number 12345 but have chosen to skip entering the AWB Date, which is a re

In [11]:
thread = {"configurable":{"thread_id":"1"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="Yes, proceed without providing it")]} , config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB 12345 and AWB Date', additional_kwargs={}, response_metadata={}, id='1412bc76-895c-4200-96fb-02fd740c78e0'),
  AIMessage(content='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', additional_kwargs={}, response_metadata={}, id='d163020b-e10a-451b-bc82-38fdfa3f2e34'),
  HumanMessage(content='Skip', additional_kwargs={}, response_metadata={}, id='31933bb7-b6de-4370-a711-b218fcd2a2d2'),
  AIMessage(content='You provided AWB number 12345 but chose to skip entering the AWB Date, which is required. Would you like to provide the AWB Date now, or should we proceed without it?', additional_kwargs={}, response_metadata={}, id='4ada157b-1dd6-4ee4-9b21-c1a328bf3c9b'),
  HumanMessage(content='Skip', additional_kwargs={}, response_metadata={}, id='262aaf5d-a0b0-4c98-9358-4fcf758affe5'),
  AIMessage(content='You have provided AWB number 12345 but have chosen to skip entering the AWB Date, which is a re

In [12]:
thread = {"configurable":{"thread_id":"1"}}
result = SupervisorAgent.invoke({"messages":[HumanMessage(content="proceed without providing")]} , config=thread)
result

{'messages': [HumanMessage(content='I want to enter the AWB 12345 and AWB Date', additional_kwargs={}, response_metadata={}, id='1412bc76-895c-4200-96fb-02fd740c78e0'),
  AIMessage(content='Please provide the AWB Date (in YYYY-MM-DD format) so we can proceed with your request.', additional_kwargs={}, response_metadata={}, id='d163020b-e10a-451b-bc82-38fdfa3f2e34'),
  HumanMessage(content='Skip', additional_kwargs={}, response_metadata={}, id='31933bb7-b6de-4370-a711-b218fcd2a2d2'),
  AIMessage(content='You provided AWB number 12345 but chose to skip entering the AWB Date, which is required. Would you like to provide the AWB Date now, or should we proceed without it?', additional_kwargs={}, response_metadata={}, id='4ada157b-1dd6-4ee4-9b21-c1a328bf3c9b'),
  HumanMessage(content='Skip', additional_kwargs={}, response_metadata={}, id='262aaf5d-a0b0-4c98-9358-4fcf758affe5'),
  AIMessage(content='You have provided AWB number 12345 but have chosen to skip entering the AWB Date, which is a re